In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFAutoModel, BertTokenizer

2023-05-14 17:07:32.564128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/joe/anaconda3/envs/sentimentAnalysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load and preprocess the data
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin', engine='python', header=None)
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df = df.sample(n=50000)

In [3]:
X = df['text']
labels = df['target']
labels[labels == 4] = 1

print(labels)

1167764    1
669817     0
797095     0
1054105    1
43220      0
          ..
1101314    1
74001      0
962867     1
569855     0
187423     0
Name: target, Length: 50000, dtype: int64


/tmp/ipykernel_3740/2086713888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[labels == 4] = 1


In [4]:
# Tokenize and pad the sequences
'''tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)'''

'tokenizer = Tokenizer()\ntokenizer.fit_on_texts(X)\nsequences = tokenizer.texts_to_sequences(X)'

In [5]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

SEQ_LEN = 50

# Tokenize the input data
input_ids = []
def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=SEQ_LEN,
        truncation=True, padding='max_length',
        add_special_tokens=True, return_attention_mask=True,
        return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [ ]:
Xids = np.zeros((len(df), SEQ_LEN))
Xmask = np.zeros((len(df), SEQ_LEN))

In [ ]:
labels = tf.keras.utils.to_categorical(labels)

In [7]:
print(input_ids[1])

[101, 2145, 4033, 1005, 1056, 9051, 3682, 1012, 1012, 1012, 10800, 1999, 2625, 2084, 2484, 2847, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
max_len = 50
#padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [22]:
# Convert labels to one-hot encoded format
#labels = tf.keras.utils.to_categorical(labels)

In [9]:
# Create the attention mask matrix
attention_mask = np.ones_like(input_ids)
padded_positions = np.equal(input_ids, 0)
attention_mask[padded_positions] = 0

In [10]:
print(attention_mask.shape)

(50000, 50)


In [11]:
dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, labels))

def map_func(sequence, attention, labels):
    return {'sequence': sequence, 'mask': attention}, labels

dataset = dataset.map(map_func)

for i in dataset.take(1):
    print(i)

2023-05-14 17:10:34.576983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 17:10:34.644607: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


({'sequence': <tf.Tensor: shape=(50,), dtype=int32, numpy=
array([  101,  1030,  2376,  7840, 15239,  1008,  2061,  1008,  5580,
        2017, 25187,  2006,  2068,  2108,  1008,  7975,  1008,  6763,
        1010,  4728,  1045,  2453,  2031,  2245,  1997,  4977, 19021,
        3334, 16820,   102,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)>, 'mask': <tf.Tensor: shape=(50,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])>}, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [12]:
dataset = dataset.batch(32)

In [13]:
ds_len = len(df)
print('dataset length: ', ds_len)

train_size = 0.8
train = dataset.take(round(ds_len*train_size))
test = dataset.skip(round(ds_len*train_size))

del dataset

dataset length:  50000


In [28]:
# Convert input data to TensorFlow tensors
'''input_ids = tf.convert_to_tensor(input_ids)
attention_mask = tf.convert_to_tensor(attention_mask)
labels = tf.convert_to_tensor(labels)'''

'input_ids = tf.convert_to_tensor(input_ids)\nattention_mask = tf.convert_to_tensor(attention_mask)\nlabels = tf.convert_to_tensor(labels)'

In [14]:
# Define the BERT model
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
# Build the model
input_seq = tf.keras.layers.Input(shape=(max_len,), name='sequence', dtype='int32')
mask = tf.keras.layers.Input(shape=(max_len,), name='mask', dtype='int32')

embeddings = bert(input_seq, attention_mask=mask)[0]

X = tf.keras.layers.MaxPooling1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(1, activation='softmax', name='output')(X)

model = tf.keras.Model(inputs=[input_seq, mask], outputs=y)

In [16]:
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['sequence[0][0]',               
                                thPoolingAndCrossAt               'mask[0][0]']                   
                                tentions(last_hidde                                               
                                n_state=(None, 50,                                            

In [21]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(0.01)
#loss = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy(name='accuracy')

model.compile(optimizer=optimizer, loss='BinaryCrossentropy', metrics=[acc])

In [22]:
history = model.fit(train,validation_data=test,epochs=3)

Epoch 1/3


ValueError: in user code:

    File "/home/joe/anaconda3/envs/sentimentAnalysis/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/joe/anaconda3/envs/sentimentAnalysis/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/joe/anaconda3/envs/sentimentAnalysis/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/joe/anaconda3/envs/sentimentAnalysis/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/joe/anaconda3/envs/sentimentAnalysis/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/home/joe/anaconda3/